In [2]:
import pandas as pd

In [3]:
df19 = pd.read_csv('../data/cfs_2019.csv')
df20 = pd.read_csv('../data/cfs_2020.csv')

In [4]:
df19.head()

,Incident_Number,Area_Occ,Rpt_Dist,Dispatch_Date,Dispatch_Time,Call_Type_Code,Call_Type_Text
0,LPD190704000047,N Hollywood,1532.0,07/04/2019 12:00:00 AM,00:14:49,507F,FIREWORKS
1,LPD190529006334,Outside,NaN,05/29/2019 12:00:00 AM,21:28:53,006,CODE 6
2,LPD190218001024,Southwest,379.0,02/18/2019 12:00:00 AM,07:46:54,507C,CONSTRUCTION
3,PD19120600001346,Northeast,1109.0,12/06/2019 12:00:00 AM,08:32:27,006,CODE 6
4,LPD190216004881,Van Nuys,933.0,02/16/2019 12:00:00 AM,21:14:49,507P,PARTY


In [6]:
df19.Dispatch_Date = pd.to_datetime(df19.Dispatch_Date, infer_datetime_format=True)
df19.head()

In [8]:
df20.Dispatch_Date = pd.to_datetime(df20.Dispatch_Date, infer_datetime_format=True)
df20.head()


,Incident_Number,Area_Occ,Rpt_Dist,Dispatch_Date,Dispatch_Time,Call_Type_Code,Call_Type_Text
0,PD20012600002716,Rampart,245.0,2020-01-26,14:37:04,211S,SUSP
1,PD20011400005678,Outside,NaN,2020-01-14,23:26:20,006,CODE 6
2,PD20011100005314,Outside,NaN,2020-01-11,23:32:40,006,CODE 6
3,PD20010100005220,Southwest,328.0,2020-01-01,22:34:29,245SN,SUSP NOW
4,PD20010500000660,Hollywood,636.0,2020-01-05,04:04:42,907PA2,POSS AMB O/D
